In [2]:
file = open('/Users/na395/Desktop/Python files/sequential_prot_mt.txt','r')
import re
protseq = file.readline().rstrip('\n')

file.close()

file = open('/Users/na395/Desktop/Python files/prot_seq_and_no_run_dna.txt','w')

#is able to count how many 3> runs of nts in total there are in seq
def number_of_runs(seq):
    total = (0)
    for each in ('ACGU'):
        total = total + len(re.findall(each + '{4,}', seq))
    return(total)

#returns GC percent content in seq
def GC_content(seq):
    GCcount=((seq.count('G')+seq.count('C'))/len(seq))
    return(GCcount)

#codondicts is a list of dicts (7), dict 0 = how many codons each aa has, then dict 1 = most used codon, dict 2 = 2nd etc. 
#get value by codondicts[x][key] where x is desired dict
#have manually swapped D, C, F codons around as is only 53 vs 47 % useage and will help gc content. P is 32 vs 31%.S and R codons also swapped though % the same

codondicts=[{'A': '4', 'C': '2', 'D': '2', 'E': '2', 'F': '2', 'G': '4', 'H': '2', 'I': '3', 'K': '2', 'L': '6', 'M': '1', 'N': '2', 'P': '4', 'Q': '2', 'R': '6', 'S': '6', 'T': '4', 'V': '4', 'W': '1', 'Y': '2'}
,{'A': 'GCC', 'C': 'UGU', 'D': 'GAU', 'E': 'GAG', 'F': 'UUU', 'G': 'GGC', 'H': 'CAC', 'I': 'AUC', 'K': 'AAG', 'L': 'CUG', 'M': 'AUG', 'N': 'AAC', 'P': 'CCU', 'Q': 'CAG', 'R': 'AGA', 'S': 'UCU', 'T': 'ACC', 'V': 'GUG', 'W': 'UGG', 'Y': 'UAC'}
,{'A': 'GCU', 'C': 'UGC', 'D': 'GAC', 'E': 'GAA', 'F': 'UUC', 'G': 'GGA', 'H': 'CAU', 'I': 'AUU', 'K': 'AAA', 'L': 'CUC', 'N': 'AAU', 'P': 'CCC', 'Q': 'CAA', 'R': 'AGG', 'S': 'UCC', 'T': 'ACA', 'V': 'GUC', 'Y': 'UAU'}
,{'A': 'GCA', 'G': 'GGG', 'I': 'AUA', 'L': 'UUG', 'P': 'CCA', 'R': 'CGG', 'S': 'AGC', 'T': 'ACU', 'V': 'GUU'}
,{'A': 'GCG', 'G': 'GGU', 'L': 'CUU', 'P': 'CCG', 'R': 'CGC', 'S': 'AGU', 'T': 'ACG', 'V': 'GUA'}
,{'L': 'CUA', 'R': 'CGA', 'S': 'UCA'}
,{'L': 'UUA', 'R': 'CGU', 'S': 'UCG'}]

dnaseq = ['NNN']

for each in protseq:
    dnaseq.append(codondicts[1][each])
dnaseq.append('NNN')
dnaseqcont= (''.join(dnaseq))



rundictnumbers={}
for each in ('ACGU'):
     rundictnumbers[each]= len(re.findall(each + '{4,}', dnaseqcont))   
print('runs '+ str(rundictnumbers))    
print(GC_content(dnaseqcont))


rundict={}
for each in ('ACGU'):
    rundict[each]= re.finditer(each + '{4,}', dnaseqcont)
    

#gets start and end posn of match and gets codon numbers. Switches first codon to second dict and evaluates if the
#new combo of codons (+1 at either side) still contains the run. If yes then moves to second codon and tries the 
#same. And then tries the third etc. If this fails mts first and second and cycles again. 
# If it runs out of loops then it means it cant be solved or flanking codons come into play.
#This script will not trial combinations of three mutations

for base in 'ACGU':

    for all in rundict[base]:

        firstcodon=((all.start()-all.start()%3)/3)
        lastcodon=(((all.end()-1)-(all.end()-1)%3)/3) #need to +1 if you want this to be part of a range



        tempcodons=[] #need tempcodons because it can be used to permanently change a codon and then trial changes of the others
        tempcodons=dnaseq[int(firstcodon)-1:int(lastcodon)+2]
        temptempcodons=tempcodons   #need temptempcodons because it can be used to try single changes of codons

        for z in '123456': #23456 because that is how many codon dictionaries there are
            if number_of_runs(''.join(temptempcodons)) == 0: #this is to stop the perm loop once the temp loop 
                break
            else:
                w=(0) #for marking which codon we are at
                tempcodons=dnaseq[int(firstcodon)-1:int(lastcodon)+2]
                for every in protseq[int(firstcodon)-1:int(lastcodon)]:
                    w = int(w)+1
                    if int(z)-1 >= int(codondicts[0][every]):
                        #it means that all the permanent codons for this amino acid have been tried 
                        continue
                    else:
                        #set that codon to perm and then assess
                        newpermcodon= (codondicts[int(z)][every])
                        tempcodons[w]=newpermcodon
                        for x in '123456': #23456 because that is how many codon dictionaries there are
                                if number_of_runs(''.join(temptempcodons)) == 0:
                                    break
                                y=(0)         #for marking which codon we are at

                                for each in protseq[int(firstcodon)-1:int(lastcodon)]:
                                    y=int(y)+1
                                    if int(x)-1 >= int(codondicts[0][each]):                            
                                        #it means that all the temporary codons for this amino acid have been tried 
                                        continue
                                    else:
                                        temptempcodons=tempcodons #this resets temptempcodons to the original
                                        newcodon = (codondicts[int(x)][each])
                                        temptempcodons[y]=newcodon
                                        if number_of_runs(''.join(temptempcodons)) == 0:
                                            break

                                        else:                             

                                            pass
                                            #it  means that the run of nts has NOT been removed
                                            #it means that further codon mts need to be attempted


        #hopefully temptempcodons can now be used to replace the original dnaseq



        if number_of_runs(''.join(temptempcodons)) == 0:
            #take these mutations and enter into original sequence
            for x in range(len(temptempcodons)-2):
                dnaseq[int(firstcodon)+int(x)]=temptempcodons[int(x)+1]


                
                
                
dnaseq.remove('NNN')
dnaseq.remove('NNN')               

dnaseqcont= (''.join(dnaseq))
rundictnumbers={}
for each in ('ACGU'):
    rundictnumbers[each]= len(re.findall(each + '{4,}', dnaseqcont))

print('runs '+ str(rundictnumbers))      
print(GC_content(dnaseqcont))

file.write(dnaseqcont.strip('N')+'\n')

file.close()

runs {'A': 0, 'C': 2, 'G': 0, 'U': 3}
0.5512437810945273
runs {'A': 0, 'C': 0, 'G': 0, 'U': 0}
0.5545545545545546


In [2]:
##### # GC content 40-60
#
#
#
#
#realised that of the most common codons only proline causes C problems as it is CCC. All the other common codons 
#cannot cause >3 Cs
#note WG will always have G{4}
#
# no 2* structure
# should not encode certain r.e.'s
# don't add or remove splice sites, check using gene splicer? keep mutagenesis away from edges...
#    
